In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install polars

In [3]:
import os
import gc
import random
from collections import defaultdict, Counter
from typing import List, Dict
import joblib
import pickle

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [4]:
product = pl.read_csv("/content/drive/MyDrive/kddcup2023/data/raw/products_train.csv")

# PRICE outlier processing

In [5]:
product = product.with_columns(
    pl.when(product['price'] == 40000000.07).then(None).otherwise(product['price']).alias("price")
)

# Assignment of handling locales



In [6]:
product.head()

id,locale,title,price,brand,color,size,model,material,author,desc
str,str,str,f64,str,str,str,str,str,str,str
"""B005ZSSN10""","""DE""","""RED DRAGON Amb…",30.95,"""RED DRAGON""",null,null,"""RDD0089""",null,null,"""Amberjacks Ste…"
"""B08PRYN6LD""","""DE""","""Simply Keto Lo…",17.9,"""Simply Keto""",null,"""750 g (1er Pac…",null,null,null,"""🌱 NATÜRLICHE S…"
"""B09MBZJ48V""","""DE""","""Sennheiser 508…",68.89,"""Sennheiser""","""Multi-Colour""","""One size""","""508377""","""Kunstleder""",null,"""3.5 MM BUCHSE …"
"""B08ZN6F26S""","""DE""","""AmyBenton Auto…",18.99,"""Amy & Benton""","""Animal Car""",null,"""2008B""","""aufziehauto 1 …",null,"""【Auto aufziehb…"
"""B094DGRV7D""","""DE""","""PLAYMOBIL - 70…",7.17,"""PLAYMOBIL""","""Nicht Zutreffe…","""OneSize""","""70522""","""Polypropylen""",null,"""Inhalt: 1 Stüc…"


In [7]:
product_id2locales = defaultdict(list)

In [8]:
ids = product["id"].to_list()
locales = product["locale"].to_list()

for id, locale in tqdm(zip(ids, locales), total=len(ids)):
    product_id2locales[id].append(locale)

100%|██████████| 1551057/1551057 [00:08<00:00, 181736.35it/s]


In [9]:
product = product.with_columns(
    pl.col("id").map_dict(product_id2locales).alias("available_locales")
)

<ipython-input-9-d4302415e382>:2: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  pl.col("id").map_dict(product_id2locales).alias("available_locales")


In [10]:
product.write_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/common/product_03.parquet")